In [7]:
import pandas as pd
import numpy as np

def process_metric_tables(metric_tables):
    def parse_tables_manual(metric_tables):
        dataframes = []
        for table in metric_tables:
            lines = table.strip().split('\n')
            headers = lines[0].split(maxsplit=2)
            data = []
            for line in lines[1:]:
                index_space = line.index(' ')
                first_split = line[:index_space].strip()
                remaining = line[index_space:].strip()
                parts = remaining.rsplit(maxsplit=2)
                if len(parts) == 3:
                    data.append([parts[0], parts[1], parts[2]])
                else:
                    data.append([parts[0], np.nan, np.nan])
            df = pd.DataFrame(data, columns=headers)
            df[headers[1]] = pd.to_numeric(df[headers[1]], errors='coerce')
            df[headers[2]] = pd.to_numeric(df[headers[2]], errors='coerce')
            dataframes.append(df)
        return dataframes

    def calculate_statistics_ordered(dataframes):
        metrics = dataframes[0]['Metric']
        result_list = []
        for metric in metrics:
            metric_data = [df[df['Metric'] == metric] for df in dataframes]
            metric_stats = {}
            for col in ['Training', 'Validation']:
                valid_data = pd.concat([md[col] for md in metric_data]).dropna()
                if not valid_data.empty:
                    mean_val = valid_data.mean()
                    std_val = valid_data.std()
                    metric_stats[col] = (mean_val, std_val)
                else:
                    metric_stats[col] = np.nan
            result_list.append((metric, metric_stats))
        return result_list

    def format_output_ordered(statistics_ordered):
        output_df = pd.DataFrame(columns=['Metric', 'Training', 'Validation'])
        for metric, values in statistics_ordered:
            row = {'Metric': metric}
            for col in ['Training', 'Validation']:
                if col in values and not pd.isna(values[col]):
                    row[col] = f"({values[col][0]:.3f}, {values[col][1]:.3f})"
                else:
                    row[col] = np.nan
            output_df = pd.concat([output_df, pd.DataFrame([row])], ignore_index=True)
        output_string = output_df.to_string(index=False)
        return output_string

    # Process the input tables
    dataframes = parse_tables_manual(metric_tables)
    statistics = calculate_statistics_ordered(dataframes)
    output_table = format_output_ordered(statistics)
    return output_table
# Example usage
metric_tables = [
    """                              Metric  Training  Validation
0                   Average Accuracy     0.760       0.655
1     Standard Deviation of Accuracy     0.223       0.146
2                      Best Accuracy     0.959       0.773
3                      Last Accuracy     0.956       0.773
4                        Overall AUC       NaN       0.489
5                   Maximum F1 Score       NaN       0.770
6                       Minimum Loss       NaN       0.757
7  Difference in Average Loss Last N       NaN       0.846
8  Standard Deviation of Loss Last N       NaN       0.119""",


"""                              Metric  Training  Validation
0                   Average Accuracy     0.629       0.552
1     Standard Deviation of Accuracy     0.227       0.154
2                      Best Accuracy     0.907       0.714
3                      Last Accuracy     0.906       0.709
4                        Overall AUC       NaN       0.512
5                   Maximum F1 Score       NaN       0.708
6                       Minimum Loss       NaN       0.796
7  Difference in Average Loss Last N       NaN       0.735
8  Standard Deviation of Loss Last N       NaN       0.212""",


"""                              Metric  Training  Validation
0                   Average Accuracy     0.663       0.601
1     Standard Deviation of Accuracy     0.227       0.163
2                      Best Accuracy     0.925       0.783
3                      Last Accuracy     0.897       0.697
4                        Overall AUC       NaN       0.597
5                   Maximum F1 Score       NaN       0.778
6                       Minimum Loss       NaN       0.856
7  Difference in Average Loss Last N       NaN       0.711
8  Standard Deviation of Loss Last N       NaN       0.125"""



    # More tables would follow here
]

result = process_metric_tables(metric_tables)
print(result)


                           Metric       Training     Validation
                 Average Accuracy (0.684, 0.068) (0.603, 0.052)
   Standard Deviation of Accuracy (0.226, 0.002) (0.154, 0.009)
                    Best Accuracy (0.930, 0.026) (0.757, 0.037)
                    Last Accuracy (0.920, 0.032) (0.726, 0.041)
                      Overall AUC            NaN (0.533, 0.057)
                 Maximum F1 Score            NaN (0.752, 0.038)
                     Minimum Loss            NaN (0.803, 0.050)
Difference in Average Loss Last N            NaN (0.764, 0.072)
Standard Deviation of Loss Last N            NaN (0.152, 0.052)
